In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import pypsa
import calendar
import re
import pickle
from pathlib import Path

from _helpers import configure_logging
import logging
logger = logging.getLogger(__name__)

from _helpers import calculate_annual_investment

In [25]:
if __name__ == "__main__":

    configure_logging(snakemake)

    supply = xr.open_dataset(snakemake.input["supply"])
    
    # Load additional components
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)
        
    network = pypsa.Network(override_component_attrs=override_component_attrs)

    network.import_from_csv_folder(snakemake.input["network"])

    scenario_year = snakemake.config["scenario"]["year"]
    
    snakemake.output[0] = Path(snakemake.output[0])
    snakemake.output[0].mkdir(parents=True)
    
    # Handle leap years by removing 29th of February
    supply = supply.sel(time=~((supply.time.dt.month == 2) & (supply.time.dt.day == 29)))

    # Load annual demand (consider overwrite option in config)
    if snakemake.config["scenario"]["synthetic_demand"].lower() == "gegis":
        demand = xr.open_dataarray(snakemake.input["demand"])
        
        # Handle leap years by removing 29th of February
        demand = demand.sel(time=~((demand.time.dt.month == 2) & (demand.time.dt.day == 29)))

        # Demand in MWh from time-series
        demand = demand.sel(region=snakemake.wildcards["from"], drop=True).sum().item()
    elif snakemake.config["scenario"]["synthetic_demand"].lower() == "custom":
        demand = pd.read_csv(snakemake.input["demand"], index_col="region")
        demand = demand.loc[snakemake.wildcards["from"],"demand [MWh]"]
    else:
        assert False, "Option in config for scenario -> synthethic_demand unknown."

    # Remove unused variables
    supply = supply.sel(region=snakemake.wildcards["from"], drop=True).fillna(0.)
    exclude_vars = list(filter(
                            lambda x: re.search("csp|density|pvrooftop|overlap", x), 
                            supply.data_vars))

    supply = supply.drop_vars(exclude_vars).drop_dims(["class_1","class_2","dim_0"])

    capacities = supply[list(filter(lambda x: "capacity" in x, supply.data_vars))]

    capacities = capacities.rename_vars({d:d.replace("capacity_","") for d in capacities})
    capacities = capacities.rename_vars({d:f"wind{d}" for d in ["offshore","onshoreA","onshoreB"]})

    # capacities from GEGIS are in GW, continue using MW
    capacities *= 1e3

    capacity_factors = supply[list(filter(lambda x: "CFtime" in x, supply.data_vars))]

    capacity_factors = capacity_factors.rename_vars({d:d.replace("CFtime_","") for d in capacity_factors})

    # Determine which generation capacities are available for export
    # and which are needed for satisfying domestic electricity demand
    # Simple heuristic:
    # 1.) Consider annual demand and generation per source+source class
    # 2.) Reserve (Exclude) sources with lowest LCoE for demand
    # 3.) Source + source class where demand and generation overlap has
    #     its generation capacitiy partially reduced based on share of overlap.

    # 1.) name mapping between dataset and cost.csv file for technologies
    mapping = {
        "windoffshore":"offwind",
        "windonshoreA":"onwind",
        "windonshoreB":"onwind",
        "pvplantA":"solar-utility",
        "pvplantB":"solar-utility",
              }

    wacc = pd.read_csv("data/wacc.csv", comment='#', index_col="region")
    wacc = wacc.loc[snakemake.wildcards["from"], snakemake.config["scenario"]["wacc"]]

    def calculate_investment(x):

        name = x.name
        cost_name = mapping[name]

        invest = calculate_annual_investment(cost_name, wacc, snakemake.input["costs"])

        return x * invest

    investment = capacities.apply(calculate_investment)

    # Calculate annual generation
    generation = (capacity_factors*capacities).fillna(0.).sum(dim="time")

    # Calculate LCoE
    lcoe = (investment/generation)

    # Convert capacities and LCoE into an easier to handle format
    df = xr.merge([
            capacities.to_array().rename("capacity").rename({"variable":"technology"}),
            lcoe.to_array().rename("lcoe").rename({"variable":"technology"}),
            generation.to_array().rename("annual generation").rename({"variable":"technology"})
         ]).to_dataframe()
    df = df.dropna().sort_values("lcoe")

    # Calculate annual generation across all sources + source classes
    df["cumulative generation"] = df.cumsum()["annual generation"]

    # Remove inf values (espc. for LCoE where generation = 0)
    df = df.replace([np.inf,-np.inf],np.nan).dropna()

    # Save LCoEs as intermediary result
    df.to_csv(snakemake.output[0]/"lcoes.csv")

    # Remaining demand overlapping into higher source class
    residual_demand = demand - df[df["cumulative generation"] <= demand].iloc[-1]["cumulative generation"]

    # 2.) Exclude sources reserved for domestic demand
    df = df[df["cumulative generation"] > demand]

    # 3.) Reduce capacitiy at overlap
    residual_demand_share = residual_demand/df.iloc[0]["annual generation"]

    df.iloc[0]["capacity"] *= (1.-residual_demand_share)

    # Add filtered list of generators to network

    carrier_mapping = {"windonshoreA":"wind","windonshoreB":"wind",
                       "windoffshore":"wind",
                       "pvplantA":"pv","pvplantB":"pv"}

    for idx, row in df.iterrows():
        network.add("Generator",
                    name=f"{idx[1]} {idx[0]}",
                    bus="electricity (exp)",
                    p_nom_max=row['capacity'],
                    p_nom_extendable=True,
                    p_max_pu=np.array(supply[f"CFtime_{idx[1]}"].sel({"class":idx[0]}, drop=True)),
                    capital_cost=calculate_annual_investment(mapping[idx[1]], wacc, snakemake.input["costs"]),
                    carrier=carrier_mapping[idx[1]]
                   )

    network.export_to_csv_folder(snakemake.output[0])
    
    supply.close()